# Reading the Data

In [10]:
from project_tiffany import *

In [5]:
from Bio import SeqIO
import numpy as np
import csv
from shared import *
from time import time
import numpy as np

In [6]:
def read_reads():
    """
    Reads all the RNA reads from reads.fa and returns a list of sequences as strings.

    :return: a list of RNA sequence reads as strings
    """
    max_id_len = 600
    reads = np.zeros((1575, 2), dtype='U{:d}'.format(max_id_len))
    i = 0
    for seq_record in SeqIO.parse("reads.fa", "fasta"):
        reads[i, 0] = seq_record.id
        reads[i, 1] = str(seq_record.seq)
        i += 1
    return reads

In [7]:
def read_genome():
    """
    Reads the genome sequence from genome.fa and return the genome sequence as a string.

    :return: a string of the genome sequence
    """
    genome = None
    for seq_record in SeqIO.parse("genome.fa", "fasta"):
        genome = str(seq_record.seq)
    return genome

In [8]:
def read_known_genes():
    """
    Reads the un/known genes, isoforms, and exons from genes.tab and constructs objects for each
    and return the list of constructed genes.

    :return: known_genes (a list of known Gene objects), unknown_genes (a list of unknown Gene objects)
    """
    known_genes, known_isoforms, known_exons = {}, {}, {}
    unknown_genes, unknown_isoforms, unknown_exons = {}, {}, {}
    with open("genes.tab") as tsv:
        for line in csv.reader(tsv, dialect="excel-tab"):
            name = line[0]
            if name == 'gene':
                known_genes[line[1]] = line[2].split(';')
            elif name == 'isoform':
                known_isoforms[line[1]] = line[2].split(';')
            elif name == 'exon':
                known_exons[line[1]] = Exon(line[1], int(line[2]), int(line[3]))
            elif name == 'unknown_gene':
                unknown_genes[line[1]] = line[2].split(';')
            elif name == 'unknown_isoform':
                unknown_isoforms[line[1]] = line[2].split(';')
            elif name == 'unknown_exon':
                unknown_exons[line[1]] = Exon(line[1], int(line[2]), int(line[3]))
    # Create the known Isoform objects
    for k in known_isoforms:
        known_isoforms[k] = Isoform(k, [known_exons[key] for key in known_isoforms[k]])
        
    # Create the UNknown Isoform objects
    for k in unknown_isoforms:
        unknown_isoforms[k] = Isoform(k, [unknown_exons[key] for key in unknown_isoforms[k]])

    # Create the known Genes objects
    for k in known_genes:
        known_genes[k] = Gene(k, [known_isoforms[key] for key in known_genes[k]])
        
    # Create the UNknown Genes objects
    for k in unknown_genes:
        unknown_genes[k] = Gene(k, [unknown_isoforms[key] for key in unknown_genes[k]])
    return known_genes, unknown_genes

In [381]:
# Main
reads = read_reads()
genome_sequence = read_genome()
known_genes, unknown_genes = read_known_genes()
known_genes_list = []
for gene in known_genes:
    known_genes_list.append(gene)

## Initialization

In [11]:
genome = genome_sequence
genes = known_genes
s = genome_sequence[::-1] + TERMINATOR
sa = get_suffix_array(s)
L = get_bwt(s, sa)
F = get_F(L)
M = get_M(F)
occ = get_occ(L)

#### Unmatched Read Indices

In [200]:
unable_to_match = [2, 3, 8, 29, 61, 84, 91, 104, 135, 154, 163, 167, 173, 179, 200, 201, 213, 218, 253, 268, 271, 276, 281, 293, 300, 322,
 323, 358, 390, 392, 393, 413, 415, 430, 438, 444, 451, 454, 467, 475, 484, 499, 501, 506, 510, 528, 568, 570, 571, 576,
 609, 618, 625, 641, 648, 650, 657, 659, 707, 710, 719, 729, 745, 753, 760, 779, 786, 827, 836, 856, 898, 905, 907, 920,
 927, 936, 937, 956, 959, 967, 979, 985, 1014, 1015, 1035, 1050, 1062, 1080, 1083, 1085, 1086, 1094, 1100, 1107, 1119,
 1134, 1142, 1150, 1153, 1161, 1181, 1193, 1196, 1212, 1224, 1229, 1232, 1252, 1253, 1266, 1276, 1281, 1320, 1323, 1325,
 1347, 1349, 1361, 1369, 1381, 1388, 1391, 1409, 1427, 1441, 1442, 1446, 1451, 1463, 1471, 1477, 1486, 1517, 1520, 1547,
 1550, 1556, 1567, 1571]

In [201]:
should_match = [8,84,91,104,167,173,213,218,253,271,276,322,323,390,392,415,430,499,510,
                528,568,625,657,707,729,745, 779,786,836,856,927,1014,1015,1035,1083,1085,
                1094,1100,1107,1119,1134,1142,1150,1181,1196,1212,1229,1252,1253,1276,1320,
                1323,1325,1349,1409,1441,1442,1446,1471,1477,1517,1547, 1550, 1567]

In [203]:
single_match = [91,104,167,173,253,271,276,322,323,390,392,415,430,499,510,
                528,568,625,707,745, 779,786,836,856,927,1014,
                1094,1107,1119,1181,1196,1212,1229,1252,1253,1276,1320,
                1323,1325,1349,1409,1442,1446,1471,1477,1517,1547, 1550, 1567]

In [204]:
double_match = [8, 84, 213, 218, 657, 729, 1015, 1035, 1083, 1085, 1100, 1134, 1142, 1150, 1441]

In [17]:
def replace_base_at_index(read_sequence, idx, base):
    length = len(read_sequence)
    return read_sequence[:idx] + base + read_sequence[idx + 1:]

In [128]:
def find_seeds(read_sequence, mismatches=6, k=3):
    len_read, seeds_len, seeds = len(read_sequence), 0, []
    if not k:
        return []
    else:
        _range, length = exact_suffix_matches(read_sequence, M, occ)
        max_so_far, updated = (_range, length, read_sequence, 0), True
        while max_so_far[3] < mismatches and max_so_far[1] < len_read and updated:
            updated = False
            possible_changes = [b for b in BASES if b != max_so_far[2][len_read - max_so_far[1] - 1]]
            misses = max_so_far[3]
            for change in possible_changes:
                new_read = replace_base_at_index(max_so_far[2], len_read - max_so_far[1] - 1, change)
#                 print('         ' + str(len_read - max_so_far[1] - 1))
                new_range, new_length = exact_suffix_matches(new_read, M, occ)
                if new_length > max_so_far[1]:
                    max_so_far = (new_range, new_length, new_read, misses + 1)
                    updated = True
                    if max_so_far[1] >= len_read:
                        break
        print('      ' + str(max_so_far))
        seeds = [([sa[i] for i in range(max_so_far[0][0], max_so_far[0][1])], max_so_far[1], max_so_far[3])]
#         seeds = [(max_so_far[0], max_so_far[1], max_so_far[3])]
        if len_read == max_so_far[1]:
            return seeds
        else:
            return seeds + find_seeds(max_so_far[2][:len(max_so_far[2]) - max_so_far[1]],
                                      mismatches=mismatches - max_so_far[3], k=k-1)

In [435]:
def check_seeds(_range, length, high_bound=None):
    valid_seeds = []
    for i in range(_range[0], _range[1]):
        location = sa[i] + length
        if not high_bound or location < high_bound and MIN_INTRON_SIZE <= high_bound - location <= MAX_INTRON_SIZE:
            valid_seeds.append(len(genome_sequence) - location)
    return valid_seeds

In [251]:
def replace_bases_at_indices(read_sequence, _range, bases):
    length = len(read_sequence)
    return read_sequence[:_range[0]] + bases + read_sequence[_range[1]:]

In [400]:
def exact_match(string):
    _range, length = exact_suffix_matches(string, M, occ)
    if length == len(string):
        return True, _range
    else:
        return False, None

In [492]:
def find_max_so_far(max_so_far, len_read, mismatches):
    if max_so_far[3] < mismatches and max_so_far[1] < len_read:        
        misses = max_so_far[3]
#         print('      ' + 'changes')
        changes = {}
        possible_changes = [b for b in BASES if b != max_so_far[2][len_read - max_so_far[1] - 1]]
        for change in possible_changes:
            new_read = replace_bases_at_indices(max_so_far[2], (len_read - max_so_far[1] - len(change),
                                                                len_read - max_so_far[1]), change)
            new_range, new_length = exact_suffix_matches(new_read, M, occ)
            changes[change] = (new_range, new_length, new_read, misses + 1)
#         print('      ' + str(len(changes)))
        for change in changes:
            test = find_max_so_far(changes[change], len_read, mismatches)
            
            if test:
                aligned, maxi = test
                if aligned:
                    return aligned, maxi
#                 else:
#                     print('      ' + str(change))
#             else:
#                 print(change)
        return False, ()
    elif max_so_far[1] == len_read and max_so_far[3] <= mismatches:
        return True, max_so_far
    else:
        return False, max_so_far

In [493]:
def seed_finder(read_sequence, high_bound=None, mismatches=6, k=2):
    len_read, seeds_len, seeds = len(read_sequence), 0, []
    if k == 1:
        _range, length = exact_suffix_matches(read_sequence, M, occ)
        max_so_far, updated = (_range, length, read_sequence, 0), True
        
        test, max_so_far = find_max_so_far(max_so_far, len_read, mismatches)
#         print('       oh ' + str(max_so_far))
        if max_so_far:
            if max_so_far[1] == len_read:
                seeds = check_seeds(max_so_far[0], max_so_far[1], high_bound)
                if seeds:
                    return True, (seeds, max_so_far[1])
                else:
                    return False, max_so_far[2]
            else:
                return False, max_so_far[2]
        return False, ()
#     else:
#         for i in range(len(read_sequence) - 1, -1, -1):
#             test_read = read_sequence[i:]
#             _range, length = exact_suffix_matches(test_read, M, occ)
#             if length == len(test_read):
                

In [474]:
reverse_read = reads[323][1][::-1]
print('        ' + str(reverse_read))
t = -time()
seedds = seed_finder(reverse_read, k=1)
print('        time: ' + str(t + time()))
print('        ' + str(390) + ": " + str(seedds))
print('        ' + str([(i, [seedds[1][i], reverse_read[i]]) for i in range(len(seedds[1]))
                          if seedds[1][i] != reverse_read[i]]))
# print('        ' + str(seedds[0] > seedds[1]))

        ATACAATATTTGATGTTTTGAGTGAGTCTGACTTAACAATTCAATGTAAC
        time: 0.4729800224304199
        390: (False, ())
        []


In [454]:
print('          ' + reads[91][1])
aligned = genome_sequence[2590477:2590477+50]
print('          ' + aligned)
print('          ' + str([(i, [aligned[i], reads[91][1][i]]) for i in range(len(aligned)) if aligned[i] != reads[91][1][i]]))

          AGGAAATTGAAAGTGTGAAAGAAAACACTGATAAACTTCTAACGGCTATC
          AGGAAATTGAAAGTGTGAAAGAAAAGACTGATAAACTTCTAAGGGCTATG
          [(25, ['G', 'C']), (42, ['G', 'C']), (49, ['G', 'C'])]


In [494]:
not_matched, total_time, unmatched_time, matched_time = 0, 0, 0, 0
lst = should_match
for i in lst:
    t = -time()
    reverse_read = reads[i][1][::-1]
#     print('        ' + str(len(reverse_read)))
    matched, match = seed_finder(reverse_read, k=1)
    total_time += time() + t
    if not matched:
        not_matched += 1
        unmatched_time += time() + t
    else:
        matched_time += time() + t
    print('        ' + str(i) + ": " + str(match))
print('        ' + str(len(lst)) + ' ' + str(not_matched))
print('        Avg Time:' + str(total_time / len(lst)) + 's')
print('        Avg Unmatched Time:' + str(unmatched_time / not_matched) + 's')
print('        Avg matched Time:' + str(matched_time / (len(lst) - not_matched)) + 's')

        8: ()
        84: ()
        91: ([2590477], 50)
        104: ([2559519], 50)
        167: ([4975316], 50)
        173: ([3920513], 50)
        213: ()
        218: ()
        253: ()
        271: ([6455465], 50)
        276: ([6455594], 50)
        322: ()
        323: ([3941393], 50)
        390: ([2574410], 50)
        392: ([6514438], 50)
        415: ()
        430: ([3941311], 50)
        499: ([2564134], 50)
        510: ([6496086], 50)
        528: ([6455479], 50)
        568: ()
        625: ()
        657: ()
        707: ()
        729: ([3923272], 50)
        745: ([2558955], 50)
        779: ()
        786: ()
        836: ()
        856: ([6455659], 50)
        927: ([6455617], 50)
        1014: ([6455495], 50)
        1015: ()
        1035: ()
        1083: ([2571829], 50)
        1085: ()
        1094: ()
        1100: ()
        1107: ()
        1119: ()
        1134: ()
        1142: ([2564522], 50)
        1150: ()
        1181: ()
        1196: ([2559675], 5

In [174]:
def return_seeds(seed_set, high_bound=len(genome_sequence)):
    if len(seed_set) == 0:
        return []
    seeds, length, _ = seed_set[0]
    if len(seed_set) == 1:
        return [(len(genome_sequence) + 1 - s - length, length) for s in seeds if s + length < high_bound]
    elif len(seed_set) == 2:
        possible_set = []
        for seed in seeds:
            returned_seeds = return_seeds(seed_set[1:], high_bound=seed)
            if returned_seeds:
                possible_set.append(([len(genome_sequence) - seed - length, returned_seeds[0]], length))
        return possible_set[0]
    else:
        possible_set = []
        for seed in seeds:
            returned_seeds = return_seeds(seed_set[1:], high_bound=seed)
#             print(returned_seeds, len(seed_set))
            if returned_seeds:
                possible_set.append(([len(genome_sequence) - seed - length, returned_seeds], length))
        return possible_set[0]

In [175]:
reverse_read = reads[271][1][::-1]
exact_suffix_matches(reverse_read, M, occ)

((5707426, 5707427), 26)

In [185]:
print('          ' + reads[8][1])
aligned = genome_sequence[6496171:6496171+22] +  genome_sequence[6514428:6514428+28]
print('          ' + genome_sequence[6496171:6496171+22] + "&" +  genome_sequence[6514428:6514428+28])
print('          ' + str([(i, [aligned[i], reads[8][1][i]]) for i in range(len(aligned)) if aligned[i] != reads[8][1][i]]))

          TAATCAGAAGGTGGATCAACTGGAAGATGTGCCTCCTCCAAAGAGCCGTA
          TAATCAGAAGGTGGATCAAGTG&GAAGATGTGCCACCTCCAAAGAGCCGTA
          [(19, ['G', 'C']), (33, ['A', 'T'])]


In [181]:
reverse_read = reads[84][1][::-1]
# print('        ' + str(len(reverse_read)))
seedds = return_seeds(find_seeds(reverse_read))
print('        ' + str(84) + ": " + str(seedds))
# print('        ' + str(seedds[0] > seedds[1]))

      ((9530222, 9530223), 20, 'GAGAAAGCTCCACCCTGTAGAAGGTTCTTGTGTATAAAATTACAAGAATA', 6)
      ((2002055, 2002056), 12, 'GAGAAAGCTCCACCCTGTAGAAGGTTCTTG', 0)
      ((4982144, 4982147), 11, 'GAGAAAGCTCCACCCTGT', 0)
        84: ([6834247, ([2669220, (3230313, 11)], 12)], 20)


In [180]:
print('        ' + reads[84][1])
print('        ' + genome_sequence[6834247:6834247+20] + '&' + genome_sequence[2669220:2669220+12] + '&' + genome_sequence[3230313:3230313+11])

        ATAAGAACATTCATCTGGTAGTTCTTGGAAGATGTCCCACCTCGAAAGAG
        ATAAGAACATTAAAATATGT&GTTCTTGGAAGA&GTCCCACCTCC


In [380]:
for i in should_match:
    reverse_read = reads[i][1][::-1]
#     print('        ' + str(len(reverse_read)))
    print('        ' + str(i) + ": " + str(find_seeds(reverse_read)))

      ((8246800, 8246801), 35, 'ATGCCGAGAAACCTCTATAGTTTGAATGGTGAACTAGGTGGAAGACTAAT', 6)
      ((3116637, 3116639), 15, 'ATGCCGAGAAACCTC', 0)
        8: [([4452794], 35, 6), ([4434544, 8323120], 15, 0)]
      ((9530222, 9530223), 20, 'GAGAAAGCTCCACCCTGTAGAAGGTTCTTGTGTATAAAATTACAAGAATA', 6)
      ((2002055, 2002056), 12, 'GAGAAAGCTCCACCCTGTAGAAGGTTCTTG', 0)
      ((4982144, 4982147), 11, 'GAGAAAGCTCCACCCTGT', 0)
        84: [([4114733], 20, 6), ([8279768], 12, 0), ([7718677, 4066902, 4305406], 11, 0)]
      ((6880151, 6880152), 50, 'GTATCGGGAATCTTCAAATAGTCAGAAAAGAAAGTGTGAAAGTTAAAGGA', 3)
        91: [([8358473], 50, 3)]
      ((593888, 593889), 50, 'AACGGCGAGGGGGACGTCGTCCCCTTCGTCACTGTCGTGGTGAACGGGTA', 1)
        104: [([8389431], 50, 1)]
      ((4319124, 4319125), 50, 'CCTCGAACGTCACTCGGCTCTAGTGCGATGACGTGAGGTCGGACCCGCTG', 5)
        167: [([5973634], 50, 5)]
      ((1210471, 1210472), 50, 'ACAAAGAGATCTACAACAGATAGTGTAGGGAAAAATTTTTACTTGTATTA', 2)
        173: [([7028437], 50, 2)]
      ((79

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [470]:
genome_length = len(genome_sequence)
known_transcriptome = {}
for gene in known_genes:
    known_transcriptome[gene] = {}
    for iso in gene.isoforms:
        isoform, exons, total_len = '', [], 0
        for ex in iso.exons:
            isoform += genome_sequence[ex.start: ex.end]
            exons.append((total_len, ex.start, ex.end - ex.start))
            total_len += ex.end - ex.start
        known_transcriptome[gene][iso] = [isoform, exons]

def find_transcriptome_alignment(read_len, align_start, exons):
    """
    Finds the alignment to the genome sequence of length read_len starting at align_start in the isoform created by
    the exons.

    :param read_len: length of the RNA read
    :param align_start: start of the alignment in the isoform created by the exons
    :param exons: the exons
    :return: alignment in the format mentioned in Aligner.align if less than or equal to number of alignment parts,
             else -1
    """
    # maximum number of un-gapped alignments
    k = 3

    def find_start_location(lo, hi):
        """
        Recursive binary search function that returns the index of the exon in which align_start lies.

        :param lo: lower index of the search
        :param hi: higher index of the search
        :return: the exon in which the start of the alignment lies
        """
        mid = (lo + hi) // 2
        if exons[mid][0] <= align_start < exons[mid][0] + exons[mid][2]:
            return mid
        elif exons[mid][0] + exons[mid][2] <= align_start:
            return find_start_location(mid + 1, hi)
        else:
            return find_start_location(lo, mid - 1)

    idx = find_start_location(0, len(exons))
    align = [(0, exons[idx][1] + (align_start - exons[idx][0]),
              read_len if read_len + (align_start - exons[idx][0]) <= exons[idx][2]
              else exons[idx][2] - (align_start - exons[idx][0]))]
    read_len -= exons[idx][2] - (align_start - exons[idx][0])
    while read_len > 0:
        idx += 1
        align.append((align[-1][0] + align[-1][2], exons[idx][1],
                      read_len if read_len <= exons[idx][2]
                      else exons[idx][2]))
        read_len -= exons[idx][2]
    return align if len(align) <= k else -1


def align_to_isoform(read, isoform, exons):
    """
    Performs alignment to the isoform element-by-element as long as it does not exceed MAX_NUM_MISMATCHES.

    :param read: RNA read sequence
    :param isoform: isoform
    :param exons: exons in the isoform
    :return: a tuple (alignment, mismatches)
        alignment: alignment to genome if one exists else -1
        mismatches: number of mismatches in the alignment if one exists otherwise float("inf")
    """
    match = (-1, float("inf"))
    for i in range(len(isoform) - len(read) + 1):
        j, mismatches = 0, 0
        while j < len(read):
            if isoform[i + j] != read[j]:
                mismatches += 1
            if mismatches > MAX_NUM_MISMATCHES:
                break
            j += 1
        if j == len(read) and mismatches < match[1]:
            match = (i, mismatches)
    return match if match[0] == -1 else (find_transcriptome_alignment(len(read), match[0], exons), match[1])


def align_to_transcriptome(read_sequence):
    """
    Performs an alignment to all the known isoforms (i.e. the annotated transcriptome) and returns the one with the
    smallest number of mismatches.

    :param read_sequence: RNA read sequence
    :return: a tuple (alignment, mismatches)
        alignment: alignment to genome if one exists else -1
        mismatches: number of mismatches in the alignment if one exists otherwise float("inf")
    """
    match = (-1, float("inf"))
    for gene in known_transcriptome:
        for iso in known_transcriptome[gene]:
            alignment, mismatches = align_to_isoform(read_sequence, known_transcriptome[gene][iso][0],
                                                             known_transcriptome[gene][iso][1])
            if alignment != -1 and mismatches <= match[1]:
                match = (alignment, mismatches)
    return match

def exact_match(string):
    """
    Returns whether or not an exact match of the string is found in the reversed genome sequence and if found, the
    range in the suffix array indicating where they start in the reversed genome.

    :param string: string to check
    :return: a tuple (match, range)
        match: a boolean indicating whether an exact match of the string exists in the reversed genome sequence
        range: None if not match, otherwise the range returned by exact_suffix_matches
    """
    _range, length = exact_suffix_matches(string, M, occ)
    if length == len(string):
        return True, _range
    else:
        return False, None

def replace_base_at_index(read_sequence, idx, base):
    """
    Returns a copy of read_sequence with the idx-th element replaced by base.

    :param read_sequence: reversed RNA read sequence
    :param idx: the index of the replacement
    :param base: the base character to replace the idx-element of read_sequence
    :return: read_sequence with the idx-th element replaced with base
    """
    return read_sequence[:idx] + base + read_sequence[idx + 1:]

def check_seeds(_range, length, high_bound=None):
    """
    Returns a valid set of seeds, i.e. starting locations in the original genome sequence.

    :param _range: range of the seed locations in the suffix array, self.sa
    :param length: length of the match for the genome suffixes starting at the index locations in the range in the
                   suffix array
    :param high_bound: bound which every index location in _range + length has to be less than
    :return: list of valid seeds
    """
    valid_seeds = []
    for i in range(_range[0], _range[1]):
        location = sa[i] + length
        if not high_bound or location < high_bound and MIN_INTRON_SIZE <= high_bound - location <= MAX_INTRON_SIZE:
            valid_seeds.append(genome_length - location)
    return valid_seeds

def find_max_so_far(max_so_far, len_read, mismatches):
    """
    A recursive function that finds (if it exists) a **single** ungapped alignment of the reversed RNA read to the
    reversed genome with the mismatches constraint. It does so by considering all the permutations of the base at
    the index of maximum alignment (second element) in max_so_far and recursively calling the function on the new
    max_so_far if there exists a match in the genome (by calling exact_match(string)). The recursive call stack
    terminates if the number of mismatches (fourth element) of max_so_far exceeds number of allowed mismatches
    (no alignment) or the length (second element) of max_so_far becomes equal to len_read (alignment!).

    :param max_so_far: the current best alignment of the reversed read sequence and the reversed genome
    :param len_read: length of the original RNA read
    :param mismatches: maximum number of mismatches allowed
    :return: a tuple (aligned, mx_so_far)
        aligned: a boolean indicating whether or not a **single** ungapped alignment was found
        mx_so_far: best max_so_far found when the terminating conditions are satisfied as documented above
    """
    if max_so_far[3] < mismatches and max_so_far[1] < len_read:
        misses = max_so_far[3]
        changes = {}
        matched, new_range = exact_match(max_so_far[2][:max_so_far[1] + 1])
        if matched:
            changes[''] = (new_range, max_so_far[1] + 1, max_so_far[2], misses)
        possible_changes = [b for b in BASES if b != max_so_far[2][max_so_far[1]]]
        for change in possible_changes:
            new_length = max_so_far[1] + 1
            new_read = replace_base_at_index(max_so_far[2], max_so_far[1], change)
            matched, new_range = exact_match(new_read[:new_length])
            if matched:
                changes[change] = (new_range, new_length, new_read, misses + 1)
        for change in changes:
            test = find_max_so_far(changes[change], len_read, mismatches)
            if test:
                aligned, maxi = test
                if aligned:
                    return aligned, maxi
        return False, max_so_far
    elif max_so_far[1] == len_read and max_so_far[3] <= mismatches:
        return True, max_so_far
    else:
        return False, max_so_far

def seed_finder(read_sequence, high_bound=None, mismatches=MAX_NUM_MISMATCHES, k=1):
    """
    Note: this function only looks for a single ungapped alignment of the READ_SEQUENCE in the genome sequence
    provided in __init__.

    Returns a tuple as documented below indicating whether an alignment has been found and if found, return a list
    of **single** tuple containing the start index of the alignment in the *actual (unreversed)* read_sequence,
    the start index of the alignment in the genome sequence, and the length of the alignment (which should be equal
    to len(read_sequence)).

    :param read_sequence: reversed RNA read sequence
    :param high_bound: the bound that all alignments need to be less than
    :param mismatches: maximum number of mismatched allowed
    :param k: maximum number of parts of alignment allowed
    :return: a tuple (aligned, seeds)
        aligned: a boolean indicating whether or not a **single** ungapped alignment was found
        seeds: [] if not aligned, otherwise a list of a single 3-element tuple containing the start of the alignment
        in *actual (unreversed)* read_sequence and the genome sequence, and the length of the alignment (which is
        equal to the length of the read_sequence)
    """
    len_read, seeds_len, seeds = len(read_sequence), 0, []
    if k == 1:
        max_so_far, updated = (None, 0, read_sequence, 0), True

        test, max_so_far = find_max_so_far(max_so_far, len_read, mismatches)
        if max_so_far:
            if max_so_far[1] == len_read:
                seeds = check_seeds(max_so_far[0], max_so_far[1], high_bound)
                if seeds:
                    return True, (seeds, max_so_far[1])
                else:
                    return False, max_so_far[2]
            else:
                return False, max_so_far[2]
        return False, ()


def align_to_genome(read_sequence):
    """
    Note: this function only looks for a single ungapped alignment of the READ_SEQUENCE in the genome sequence
    provided in __init__.

    Returns a tuple as documented below indicating whether an alignment has been found and if found, return a list
    of **single** tuple containing the start index of the alignment in the READ_SEQUENCE, the start index of the
    alignment in the genome sequence, and the length of the alignment (which should be equal to len(read_sequence)).

    :param read_sequence: RNA read sequence
    :return: a tuple (aligned, seeds)
        aligned: a boolean indicating whether or not a **single** ungapped alignment was found
        seeds: [] if not aligned, otherwise a list of a single 3-element tuple containing the start of the alignment
        in read_sequence and the genome sequence, and the length of the alignment (which is equal to the length of
        the read_sequence)
    """
    reverse_read_sequence = read_sequence[::-1]
    aligned, seeds = seed_finder(reverse_read_sequence)
    if aligned:
        return True, [(0, seeds[0][0], 50)]
    else:
        return False, []


def align(read_sequence):
    """
    Returns an alignment to the genome sequence. An alignment is a list of pieces.
    Each piece consists of a start index in the read, a start index in the genome, and a length
    indicating how many bases are aligned in this piece. Note that mismatches are count as "aligned".

    Note that <read_start_2> >= <read_start_1> + <length_1>. If your algorithm produces an alignment that
    violates this, we will remove pieces from your alignment arbitrarily until consecutive pieces
    satisfy <read_start_2> >= <read_start_1> + <length_1>

    Return value must be in the form (also see the Project.pdf):
    [(<read_start_1>, <reference_start_1, length_1), (<read_start_2>, <reference_start_2, length_2), ...]

    If no good matches are found: return the best match you can find or return []

    Time limit: 0.5 seconds per read on average on the provided data.
    """
    alignment, mismatches = align_to_transcriptome(read_sequence)
    if alignment != -1:
        return alignment
    else:
        aligned, alignment = align_to_genome(read_sequence)
        if aligned:
            return alignment
        else:
            return []

In [471]:
align(reads[104][1])

[(0, 2559519, 50)]

In [52]:
print('          ' + reads[104][1][:43])
print('          ' + genome_sequence[2590477:2590477+43])

          ATGGGCAAGTGGTGCTGTCACTGCTTCCACTGCTGCAGGGGTA
          AGGAAATTGAAAGTGTGAAAGAAAAGACTGATAAACTTCTAAG
